# BCAUS

__Reference__: [Belthangady et al, Minimizing bias in massive multi-arm observational studies with BCAUS: balancing covariates automatically using supervision, BMC Medical Research Methodology 2021](https://bmcmedresmethodol.biomedcentral.com/articles/10.1186/s12874-021-01383-x)

## BCAUS on IHDP

In [1]:
import logging, sys
from causalforge.model import Model , PROBLEM_TYPE
from causalforge.data_loader import DataLoader 

# logger 
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

# load IHDP dataset 
r = DataLoader.get_loader('IHDP').load()
X_tr, T_tr, YF_tr, YCF_tr, mu_0_tr, mu_1_tr, X_te, T_te, YF_te, YCF_te, mu_0_te, mu_1_te = r

# model params 
params={}
params['random_state']= 271
params['learning_rate_init']= 0.001
params['nu']= 1
params['max_iter']= 100
params['alpha']= 0.0
params['dropout']= 0
params['eps']= 1e-5
params['n_iter_no_change']= 10
params['balance_threshold']= 0.1
params['verbose'] = True 
params['logger'] = logging.getLogger("BCAUS_DR")

# model 
bcaus_dr = Model.create_model("bcaus_dr",
                               params,
                               problem_type=PROBLEM_TYPE.CAUSAL_TREATMENT_EFFECT_ESTIMATION, 
                               multiple_treatments=False)



/home/jovyan/my-conda-envs/causalforge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from causalforge.metrics import eps_ATE_diff, PEHE_with_ite
import numpy as np

experiment_ids = [1,10,400]

eps_ATE_tr, eps_ATE_te = [], []
eps_PEHE_tr, eps_PEHE_te = [] , [] 



for idx in experiment_ids:    
    t_tr, y_tr, x_tr, mu0tr, mu1tr = T_tr[:,idx] , YF_tr[:,idx], X_tr[:,:,idx], mu_0_tr[:,idx], mu_1_tr[:,idx] 
    t_te, y_te, x_te, mu0te, mu1te = T_te[:,idx] , YF_te[:,idx], X_te[:,:,idx], mu_0_te[:,idx], mu_1_te[:,idx]  
    
    
    # Train your causal method on train-set ...
    bcaus_dr.fit(x_tr,t_tr,y_tr)

    # Validate your method test-set ... 
    ATE_truth_tr = (mu1tr - mu0tr).mean()
    ATE_truth_te = (mu1te - mu0te).mean()
    
    ITE_truth_tr = (mu1tr - mu0tr)
    ITE_truth_te = (mu1te - mu0te)
    
    eps_ATE_tr.append( eps_ATE_diff( bcaus_dr.predict_ate(x_tr,t_tr,y_tr), ATE_truth_tr) )
    eps_ATE_te.append( eps_ATE_diff( bcaus_dr.predict_ate(x_te,t_te,y_te), ATE_truth_te) )
        

2023-05-30 21:04:13,388 - BCAUS_DR - INFO - Epoch =0: Propensity Loss =0.5607155561447144, Covariate Loss =0.013271915726363659, Balanced covs =9
2023-05-30 21:04:13,658 - BCAUS_DR - INFO - Epoch =50: Propensity Loss =0.48983609676361084, Covariate Loss =0.0003960996982641518, Balanced covs =23
2023-05-30 21:04:14,116 - BCAUS_DR - INFO - Number of balanced covariates at end of training:23
2023-05-30 21:04:20,857 - BCAUS_DR - INFO - Epoch =0: Propensity Loss =0.5610273480415344, Covariate Loss =0.012001271359622478, Balanced covs =10
2023-05-30 21:04:21,149 - BCAUS_DR - INFO - Epoch =50: Propensity Loss =0.4866325259208679, Covariate Loss =0.0004617059894371778, Balanced covs =21
2023-05-30 21:04:21,372 - BCAUS_DR - INFO - Number of balanced covariates at end of training:24
2023-05-30 21:04:21,444 - BCAUS_DR - INFO - Epoch =0: Propensity Loss =0.5617637634277344, Covariate Loss =0.011508171446621418, Balanced covs =9
2023-05-30 21:04:21,582 - BCAUS_DR - INFO - Epoch =50: Propensity Loss

## Results 

In [3]:
bcaus_dr.support_ite()

False

In [4]:
import pandas as pd 

pd.DataFrame([[np.mean(eps_ATE_tr),np.mean(eps_ATE_te)]],
             columns=['eps_ATE_tr','eps_ATE_te'], 
             index=['BCAUS_DR'])

,eps_ATE_tr,eps_ATE_te
BCAUS_DR,0.012826,0.260861
